In [1]:
# Autoreload possibly interferes with IntelliJ debugging
# %reload_ext autoreload
# %autoreload 2
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
log = lambda msg: logging.info(msg)


# Post-human pipeline (single file)
For testing and adding to the pipeline.

In [24]:
import pandas as pd
import os

input_dir = "C:\\dev\\play\\brainwave-data"
stats_df = pd.read_csv(input_dir + os.path.sep + "stats.csv")

# Post-human processing
This requires user interaction first to provide sleep times etc.

In [3]:
from sleep_events import load_days_data

days_data = load_days_data(True)

In [18]:
import run_feature_pipeline
from run_post_human_pipeline import cached_post_human_pipeline

#dir_name = "2024-11-01-21-32-22"
dir_name = "2024-11-08-21-16-53"
input_file = f"C:\\dev\\play\\brainwave-data\\{dir_name}\\raw.fif"
yasa_df, cached = run_feature_pipeline.cached_pipeline(log, input_file, False)
#post_human_df = cached_post_human_pipeline(log, dir_name, input_file, stats_df, days_data, yasa_df)
#post_human_df

2024-12-16 09:03:31,011 - INFO - Loading cached file C:\dev\play\brainwave-data\2024-11-08-21-16-53\raw.yasa.csv


In [19]:
yasa_df

,Stage,Confidence,Epoch,Timestamp,Source,Fpz-M1_Stage,Fpz-M1_Confidence,StageInt
epoch,,,,,,,,
0,W,0.997831,0,2024-11-08 21:16:53.214781046+00:00,Fpz-M1,W,0.997831,0
1,W,0.998544,1,2024-11-08 21:17:23.214781046+00:00,Fpz-M1,W,0.998544,0
2,W,0.999779,2,2024-11-08 21:17:53.214781046+00:00,Fpz-M1,W,0.999779,0
3,W,0.999908,3,2024-11-08 21:18:23.214781046+00:00,Fpz-M1,W,0.999908,0
4,W,0.999828,4,2024-11-08 21:18:53.214781046+00:00,Fpz-M1,W,0.999828,0
...,...,...,...,...,...,...,...,...
1133,W,0.869996,1133,2024-11-09 06:43:23.214781046+00:00,Fpz-M1,W,0.869996,0
1134,W,0.981264,1134,2024-11-09 06:43:53.214781046+00:00,Fpz-M1,W,0.981264,0
1135,W,0.961904,1135,2024-11-09 06:44:23.214781046+00:00,Fpz-M1,W,0.961904,0


In [9]:
from models.eeg_states.eeg_states import load_and_prepare_settling_eeg_state_events

tired_wired_eeg_state_events = load_and_prepare_settling_eeg_state_events()

C:\Users\graha\.conda\envs\tf\lib\site-packages\google\cloud\firestore_v1\base_collection.py:300: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
C:\dev\play\brainwave-processor\models\eeg_states\eeg_states.py:130: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2024-10-09 20:17:27.992000+00:00' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  events.at[i, 'since'] = since_time
C:\dev\play\brainwave-processor\models\eeg_states\eeg_states.py:132: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  events['duration'] = events['timestamp'] - events['since']


# Replicate the pipeline

In [25]:
import os
from catboost import CatBoostClassifier
from run_post_human_pipeline import add_periods
from datetime import datetime

import run_feature_pipeline
from models.eeg_states.eeg_states import load_and_prepare_settling_eeg_state_events, add_event_type
from models.eeg_states.eeg_states_model import predict_only_tired_vs_wired_model_pipeline
from sleep_events import convert_timestamp_to_uk, convert_timestamps_to_uk
import pandas as pd

convert_timestamps_to_uk(yasa_df, 'Timestamp', 'TimestampUK')
yasa_df = add_periods(log, dir_name, input_file, stats_df, days_data, yasa_df)
add_event_type(yasa_df, tired_wired_eeg_state_events)

# TiredVsWired model
# before_ready_to_sleep = yasa_df[yasa_df['60MinsBeforeReadyToSleep'] == 1]
# models_and_data_before_ready_to_sleep = [predict_only_tired_vs_wired_model_pipeline('main', before_ready_to_sleep, False)]
models_and_data = [predict_only_tired_vs_wired_model_pipeline('main', yasa_df, False)]

model = CatBoostClassifier()
model.load_model("models/eeg_states/settling-tired-vs-wired-non-realtime_catboost_model.cbm")
predictions = model.predict_proba(models_and_data[0].X)
predictions_df = pd.DataFrame(predictions, index=models_and_data[0].X.index)
# predictions_df['TiredVsWired60MinsBeforeReadyToSleepPrediction'] = predictions_df[1]
predictions_df['SettlingTiredVsWiredPrediction'] = predictions_df[1]
predictions_df.drop([0, 1], axis=1, inplace=True)
yasa_df_with_predictions = pd.concat([yasa_df, predictions_df], axis=1)

model.load_model("models/eeg_states/settling-score-non-realtime_catboost_model.cbm")
predictions = model.predict_proba(models_and_data[0].X)
predictions_df = pd.DataFrame(predictions, index=models_and_data[0].X.index)
#predictions_df['TiredVsWired60MinsBeforeReadyToSleepPrediction'] = predictions_df[1]
predictions_df['SettlingScorePrediction'] = predictions_df[1]
predictions_df.drop([0, 1], axis=1, inplace=True)
yasa_df_with_predictions = pd.concat([yasa_df_with_predictions, predictions_df], axis=1)

model.load_model("models/eeg_states/settling-v4-score-non-realtime_catboost_model.cbm")
predictions = model.predict_proba(models_and_data[0].X)
predictions_df = pd.DataFrame(predictions, index=models_and_data[0].X.index)
predictions_df['SettlingV4ScorePrediction'] = predictions_df[1]
predictions_df.drop([0, 1], axis=1, inplace=True)
yasa_df_with_predictions = pd.concat([yasa_df_with_predictions, predictions_df], axis=1)


NameError: name 'days_data' is not defined

In [29]:
convert_timestamps_to_uk(yasa_df, 'Timestamp', 'TimestampUK')

yasa_df

,Stage,Confidence,Epoch,Timestamp,Source,Fpz-M1_Stage,Fpz-M1_Confidence,StageInt,TimestampUK
epoch,,,,,,,,,
0,W,0.997831,0,2024-11-08 21:16:53.214781046+00:00,Fpz-M1,W,0.997831,0,2024-11-08 21:16:53.214781046+00:00
1,W,0.998544,1,2024-11-08 21:17:23.214781046+00:00,Fpz-M1,W,0.998544,0,2024-11-08 21:17:23.214781046+00:00
2,W,0.999779,2,2024-11-08 21:17:53.214781046+00:00,Fpz-M1,W,0.999779,0,2024-11-08 21:17:53.214781046+00:00
3,W,0.999908,3,2024-11-08 21:18:23.214781046+00:00,Fpz-M1,W,0.999908,0,2024-11-08 21:18:23.214781046+00:00
4,W,0.999828,4,2024-11-08 21:18:53.214781046+00:00,Fpz-M1,W,0.999828,0,2024-11-08 21:18:53.214781046+00:00
...,...,...,...,...,...,...,...,...,...
1133,W,0.869996,1133,2024-11-09 06:43:23.214781046+00:00,Fpz-M1,W,0.869996,0,2024-11-09 06:43:23.214781046+00:00
1134,W,0.981264,1134,2024-11-09 06:43:53.214781046+00:00,Fpz-M1,W,0.981264,0,2024-11-09 06:43:53.214781046+00:00
1135,W,0.961904,1135,2024-11-09 06:44:23.214781046+00:00,Fpz-M1,W,0.961904,0,2024-11-09 06:44:23.214781046+00:00


In [31]:

# Define the time range
start_time = yasa_df['TimestampUK'].iloc[0].strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'
end_time = yasa_df['TimestampUK'].iloc[-1].strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'
start_time

'2024-11-08T21:16:53.214Z'

In [ ]:
yasa_df

In [43]:
from datetime import datetime, timedelta
import pytz

def calculate_offset(start_time, interval_seconds):
    # Parse start_time as a timezone-aware datetime object
    # start_time = datetime.fromisoformat(start_time_str.replace("Z", "+00:00"))

    # Convert start_time to seconds since epoch
    epoch_seconds = int(start_time.timestamp())

    # Calculate misalignment
    misalignment = epoch_seconds % interval_seconds
    return (start_time.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z', misalignment)


epoch_length_secs = 30
start_time, misalignment = calculate_offset(yasa_df['TimestampUK'].iloc[0], epoch_length_secs)
end_time, misalignment = calculate_offset(yasa_df['TimestampUK'].iloc[-1], epoch_length_secs)
# Influx will return buckets starting from nearest round 30s without misalignment adjustment
group_by = f'GROUP BY time({epoch_length_secs}s, {misalignment}s)'
print (start_time, end_time, misalignment)

2024-11-08T21:16:53.214Z 2024-11-09T06:45:23.214Z 23


In [48]:
from influxdb import InfluxDBClient
import dotenv
import os
dotenv.load_dotenv()
import pandas as pd

# host="95.111.246.213"
# port=8086

# host = os.getenv('INFLUXDB_HOST')
# port = os.getenv('INFLUXDB_PORT')
username = os.getenv('INFLUXDB_USERNAME')
password = os.getenv('INFLUXDB_PASSWORD')
database = os.getenv('INFLUXDB_DATABASE')

print(host)
print(port)

client = InfluxDBClient(host=host, port=port, username=username, password=password, database=database, ssl=True, verify_ssl=False)

# Define the time range
# start_time = yasa_df['Timestamp'].iloc[0]
# end_time = yasa_df['Timestamp'].iloc[-1]

# start_time = '2024-12-14T21:04:54.099Z'
# end_time = '2024-12-15T08:13:00Z'

# Queries for each measurement
queries = {
    "android_hr": f'''
        SELECT median("hr") AS HR, median("hrvRMSSDSomewhatRecent") AS HrvRMSSDSomewhatRecent
        FROM "XL"."autogen"."android_hr"
        WHERE time >= '{start_time}' AND time <= '{end_time}'
        {group_by}
        ORDER BY time
    ''',
    "android_temp": f'''
        SELECT median("temp") AS Temp
        FROM "XL"."autogen"."android_temp"
        WHERE time >= '{start_time}' AND time <= '{end_time}'
        {group_by}
        ORDER BY time
    ''',
    "android_o2": f'''
        SELECT median("o2") AS O2,
        median("movement") as Movement
        FROM "XL"."autogen"."android_o2"
        WHERE time >= '{start_time}' AND time <= '{end_time}'
        {group_by}
        ORDER BY time
    ''',
    "android_accel": f'''
        SELECT
            min("x") as PositionMinX, max("x") as PositionMaxX, median("x") as PositionMedX,
            min("y") as PositionMinY, max("y") as PositionMaxY, median("y") as PositionMedY,
            min("z") as PositionMinZ, max("z") as PositionMaxZ, median("z") as PositionMedZ
        FROM "XL"."autogen"."android_accel"
        WHERE time >= '{start_time}' AND time <= '{end_time}'
        {group_by}
        ORDER BY time
    '''
}

# Fetch data and convert to DataFrames
dataframes = {}
for key, query in queries.items():
    result = client.query(query)
    points = list(result.get_points())
    df = pd.DataFrame(points)
    if 'time' in df:
        df['time'] = pd.to_datetime(df['time'])  # Ensure time is in datetime format
        df.set_index('time', inplace=True)
    dataframes[key] = df

# Merge all DataFrames on time index
merged_df = pd.concat(dataframes.values(), axis=1)
merged_df


95.111.246.213
8086


C:\Users\graha\.conda\envs\tf\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '95.111.246.213'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\graha\.conda\envs\tf\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '95.111.246.213'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\graha\.conda\envs\tf\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '95.111.246.213'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\graha\.conda\envs\tf\lib\site-package

,HR,HrvRMSSDSomewhatRecent,Temp,o2,Movement,PositionMinX,PositionMaxX,PositionMedX,PositionMinY,PositionMaxY,PositionMedY,PositionMinZ,PositionMaxZ,PositionMedZ
time,,,,,,,,,,,,,,
2024-11-08 21:16:53+00:00,75.0,24.0,36.55,NaN,NaN,-963.0,-891.0,-953.0,-121.0,-36.0,-101.5,123.0,369.0,136.5
2024-11-08 21:17:23+00:00,78.5,24.5,36.55,NaN,NaN,-966.0,-930.0,-950.0,-308.0,-84.0,-119.0,70.0,157.0,123.0
2024-11-08 21:17:53+00:00,84.0,53.0,36.56,NaN,NaN,-956.0,-904.0,-917.5,-263.0,0.0,-176.0,50.0,187.0,152.0
2024-11-08 21:18:23+00:00,80.0,148.0,36.57,NaN,NaN,-898.0,-646.0,-770.5,-422.0,326.0,-155.0,-365.0,673.0,489.5
2024-11-08 21:18:53+00:00,72.0,148.0,36.57,NaN,NaN,-726.0,-637.0,-713.0,-176.0,-144.0,-156.0,587.0,696.0,625.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-09 06:43:23+00:00,73.0,25.0,NaN,92.0,0.0,-79.0,-24.0,-67.0,71.0,80.0,77.5,947.0,964.0,957.5
2024-11-09 06:43:53+00:00,75.0,13.0,NaN,NaN,NaN,-81.0,-59.0,-79.0,76.0,81.0,78.0,954.0,961.0,955.0
2024-11-09 06:44:23+00:00,78.5,13.0,NaN,NaN,NaN,-81.0,-40.0,-68.0,75.0,81.0,79.5,948.0,962.0,956.0


In [41]:
dataframes.keys()

dict_keys(['android_hr', 'android_temp', 'android_o2', 'android_accel'])

In [48]:
dataframes['android_temp']

""


In [50]:
merged_df

,HR,HrvRMSSDSomewhatRecent,Temp,o2,Movement,PositionMinX,PositionMaxX,PositionMedX,PositionMinY,PositionMaxY,PositionMedY,PositionMinZ,PositionMaxZ,PositionMedZ
0,75.0,24.0,36.55,NaN,NaN,-963.0,-891.0,-953.0,-121.0,-36.0,-101.5,123.0,369.0,136.5
1,78.5,24.5,36.55,NaN,NaN,-966.0,-930.0,-950.0,-308.0,-84.0,-119.0,70.0,157.0,123.0
2,84.0,53.0,36.56,NaN,NaN,-956.0,-904.0,-917.5,-263.0,0.0,-176.0,50.0,187.0,152.0
3,80.0,148.0,36.57,NaN,NaN,-898.0,-646.0,-770.5,-422.0,326.0,-155.0,-365.0,673.0,489.5
4,72.0,148.0,36.57,NaN,NaN,-726.0,-637.0,-713.0,-176.0,-144.0,-156.0,587.0,696.0,625.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133,73.0,25.0,NaN,92.0,0.0,-79.0,-24.0,-67.0,71.0,80.0,77.5,947.0,964.0,957.5
1134,75.0,13.0,NaN,NaN,NaN,-81.0,-59.0,-79.0,76.0,81.0,78.0,954.0,961.0,955.0
1135,78.5,13.0,NaN,NaN,NaN,-81.0,-40.0,-68.0,75.0,81.0,79.5,948.0,962.0,956.0
1136,81.0,14.0,NaN,NaN,NaN,-80.0,-13.0,-67.5,76.0,80.0,78.0,951.0,969.0,955.0


In [36]:
start_time

'2024-11-08T21:16:53.214Z'

In [49]:
yasa_df.reset_index(drop=True, inplace=True)
merged_df.reset_index(drop=True, inplace=True)
pd.DataFrame.merge(yasa_df, merged_df, left_index=True, right_index=True, suffixes=['', '_r'])

,Stage,Confidence,Epoch,Timestamp,Source,Fpz-M1_Stage,Fpz-M1_Confidence,StageInt,TimestampUK,HR,...,Movement,PositionMinX,PositionMaxX,PositionMedX,PositionMinY,PositionMaxY,PositionMedY,PositionMinZ,PositionMaxZ,PositionMedZ
0,W,0.997831,0,2024-11-08 21:16:53.214781046+00:00,Fpz-M1,W,0.997831,0,2024-11-08 21:16:53.214781046+00:00,75.0,...,NaN,-963.0,-891.0,-953.0,-121.0,-36.0,-101.5,123.0,369.0,136.5
1,W,0.998544,1,2024-11-08 21:17:23.214781046+00:00,Fpz-M1,W,0.998544,0,2024-11-08 21:17:23.214781046+00:00,78.5,...,NaN,-966.0,-930.0,-950.0,-308.0,-84.0,-119.0,70.0,157.0,123.0
2,W,0.999779,2,2024-11-08 21:17:53.214781046+00:00,Fpz-M1,W,0.999779,0,2024-11-08 21:17:53.214781046+00:00,84.0,...,NaN,-956.0,-904.0,-917.5,-263.0,0.0,-176.0,50.0,187.0,152.0
3,W,0.999908,3,2024-11-08 21:18:23.214781046+00:00,Fpz-M1,W,0.999908,0,2024-11-08 21:18:23.214781046+00:00,80.0,...,NaN,-898.0,-646.0,-770.5,-422.0,326.0,-155.0,-365.0,673.0,489.5
4,W,0.999828,4,2024-11-08 21:18:53.214781046+00:00,Fpz-M1,W,0.999828,0,2024-11-08 21:18:53.214781046+00:00,72.0,...,NaN,-726.0,-637.0,-713.0,-176.0,-144.0,-156.0,587.0,696.0,625.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133,W,0.869996,1133,2024-11-09 06:43:23.214781046+00:00,Fpz-M1,W,0.869996,0,2024-11-09 06:43:23.214781046+00:00,73.0,...,0.0,-79.0,-24.0,-67.0,71.0,80.0,77.5,947.0,964.0,957.5
1134,W,0.981264,1134,2024-11-09 06:43:53.214781046+00:00,Fpz-M1,W,0.981264,0,2024-11-09 06:43:53.214781046+00:00,75.0,...,NaN,-81.0,-59.0,-79.0,76.0,81.0,78.0,954.0,961.0,955.0
1135,W,0.961904,1135,2024-11-09 06:44:23.214781046+00:00,Fpz-M1,W,0.961904,0,2024-11-09 06:44:23.214781046+00:00,78.5,...,NaN,-81.0,-40.0,-68.0,75.0,81.0,79.5,948.0,962.0,956.0
1136,W,0.814729,1136,2024-11-09 06:44:53.214781046+00:00,Fpz-M1,W,0.814729,0,2024-11-09 06:44:53.214781046+00:00,81.0,...,NaN,-80.0,-13.0,-67.5,76.0,80.0,78.0,951.0,969.0,955.0
